# Point Cloud

## Intrinsic K


In [18]:
%pip uninstall pytorch3d

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
sys.path.append("../packnet-sfm/")

from packnet_sfm.models.model_wrapper import ModelWrapper
from packnet_sfm.datasets.augmentations import resize_image, to_tensor
from packnet_sfm.utils.horovod import hvd_init, rank, world_size, print0
from packnet_sfm.utils.image import load_image
from packnet_sfm.utils.config import parse_test_file
from packnet_sfm.utils.load import set_debug
from packnet_sfm.utils.depth import write_depth, inv2depth, viz_inv_depth
from packnet_sfm.utils.logging import pcolor
from packnet_sfm.geometry.pose_utils import euler2mat

import json
import argparse
import numpy as np
import os
import torch
from glob import glob
# from pytorch3d import transforms 

In [42]:
def is_image(file, ext=('.png', '.jpg',)):
    """Check if a file is an image with certain extensions"""
    return file.endswith(ext)

def parse_args():
    parser = argparse.ArgumentParser(description='PackNet-SfM inference of depth maps from images')
    parser.add_argument('--checkpoint', type=str, help='Checkpoint (.ckpt)')
    parser.add_argument('--input', type=str, help='Input file or folder')
    parser.add_argument('--output', type=str, help='Output file or folder')
    parser.add_argument('--image_shape', type=int, nargs='+', default=None,
                        help='Input and output image shape '
                             '(default: checkpoint\'s config.datasets.augmentation.image_shape)')
    parser.add_argument('--half', action="store_true", help='Use half precision (fp16)')
    parser.add_argument('--save', type=str, choices=['npz', 'png'], default=None,
                        help='Save format (npz or png). Default is None (no depth map is saved).')
    args = parser.parse_args()
    assert args.checkpoint.endswith('.ckpt'), \
        'You need to provide a .ckpt file as checkpoint'
    assert args.image_shape is None or len(args.image_shape) == 2, \
        'You need to provide a 2-dimensional tuple as shape (H,W)'
    assert (is_image(args.input) and is_image(args.output)) or \
           (not is_image(args.input) and not is_image(args.input)), \
        'Input and output must both be images or folders'
    return args


@torch.no_grad()
def infer_and_save_depth(input_file, output_file, model_wrapper, image_shape, half, save):
    """
    Process a single input file to produce and save visualization
    Parameters
    ----------
    input_file : str
        Image file
    output_file : str
        Output file, or folder where the output will be saved
    model_wrapper : nn.Module
        Model wrapper used for inference
    image_shape : Image shape
        Input image shape
    half: bool
        use half precision (fp16)
    save: str
        Save format (npz or png)
    """
    if not is_image(output_file):
        # If not an image, assume it's a folder and append the input name
        os.makedirs(output_file, exist_ok=True)
        output_file = os.path.join(output_file, os.path.basename(input_file))

    # change to half precision for evaluation if requested
    dtype = torch.float16 if half else None

    # Load image
    image = load_image(input_file)

    # Resize and to tensor
    image = resize_image(image, image_shape)
    image = to_tensor(image).unsqueeze(0)

    # Send image to GPU if available
    if torch.cuda.is_available():
        image = image.to('cuda:{}'.format(rank()), dtype=dtype)
    
    # Depth inference (returns predicted inverse depth)
    pred_inv_depth = model_wrapper.depth(image)['inv_depths'][0]

    if save == 'npz' or save == 'png':
        # Get depth from predicted depth map and save to different formats
        filename = '{}.{}'.format(os.path.splitext(output_file)[0], save)
        print('Saving {} to {}'.format(
            pcolor(input_file, 'cyan', attrs=['bold']),
            pcolor(filename, 'magenta', attrs=['bold'])))
        write_depth(filename, depth=inv2depth(pred_inv_depth))
        # print(inv2depth(pred_inv_depth))
        
    else:
        # Prepare RGB image
        rgb = image[0].permute(1, 2, 0).detach().cpu().numpy() * 255
        # Prepare inverse depth
        viz_pred_inv_depth = viz_inv_depth(pred_inv_depth[0]) * 255
        # Concatenate both vertically
        # image = np.concatenate([rgb, viz_pred_inv_depth], 0)
        image = viz_pred_inv_depth
        # Save visualization
        print('Saving {} to {}'.format(
            pcolor(input_file, 'cyan', attrs=['bold']),
            pcolor(output_file, 'magenta', attrs=['bold'])))
 

def get_images(filename, imgshp):
    image = load_image(filename)
    image = resize_image(image,imgshp)
    image = to_tensor(image).unsqueeze(0)

    if torch.cuda.is_available():
        imgae = image.to("cuda:{}".format(rank()),dtype=None)

    return image

poses = dict()

@torch.no_grad()
def infer_and_save_pose(input_file, input_refs, model_wrapper,image_shape):
    '''
    Output Pose
    input_file    : input_file
    input_refs    : reference frame 
    model_wrapper : nn module
    '''
    basename = os.path.basename(input_file)
    
    image_ref = [get_images(input_ref,image_shape) for input_ref in input_refs]
    image = get_images(input_file,image_shape)

    # 1st to 2nd img pose
    pose = model_wrapper.pose(image,image_ref)[0][0] 
    angle = pose[3:]
    angle = torch.flip(angle,[0])
    angle = angle.reshape(1,-1)
    rot_matrix = euler2mat(angle)
    rot_matrix.reshape(3,3)
    translation = pose[:3]

    poses[basename] = (rot_matrix,translation)
    

In [24]:
x = torch.tensor([[[1,3,4],
                [2,3,4],[3,4,5]]])
x.reshape(3,3)


tensor([[1, 3, 4],
        [2, 3, 4],
        [3, 4, 5]])

Parameters

### Image to world coordinate

In [55]:
import sys
sys.path.append("../camviz/")
from packnet_sfm.geometry.camera_utils import construct_K
import cv2
# from packnet_sfm.geometry.camera import Camera
import camviz as cv 
import numpy as np

# depth
fx=9.047872e+02
fy=9.017079e+02
cx=6.946163e+02 
cy=2.353088e+02 
K = construct_K(fx,fy,cx,cy)

np.savetxt("../Kitti/3_pose/intrinsic.txt", K.numpy())




### main

In [43]:

sys.path.append("../Kitti/")
ckpt = "../packnet-sfm/PackNet01_HR_velsup_CStoK.ckpt"
imgshp = (384,1280)

inputs="../Kitti/3"  
depth_path="../Kitti/3_depth/"
outputs="../Kitti/3_pose/pose.json"


In [48]:
# main:

hvd_init()
config, state_dict = parse_test_file(ckpt)
print(config.datasets.augmentation.image_shape)

set_debug(config.debug)

# Model Wrapper
model_wrapper = ModelWrapper(config,load_datasets=False)
model_wrapper.load_state_dict(state_dict)

if torch.cuda.is_available():
    model_wrapper = model_wrapper.to('cuda:{}'.format(rank()),dtype=None)
model_wrapper.eval()


if os.path.isdir(inputs):
    # If input file is a folder, search for image files
    files = []
    for ext in ['png', 'jpg']:
        files.extend(glob((os.path.join(inputs, '*.{}'.format(ext)))))
    files.sort()
    print('Found {} files'.format(len(files)))
else:
    # Otherwise, use it as is
    files = [inputs]

# infer_and_save_pose(
#     input_file=files[0],input_refs=input_ref,output_file=outputs,
#     model_wrapper=model_wrapper,image_shape=imgshp)


for i in range(1,len(files[rank()::world_size()])-1):
    input_file = files[i]
    input_ref = [files[i-1],files[i+1]]
    infer_and_save_pose(
        input_file, input_ref, model_wrapper, imgshp)




(384, 1280)
### Preparing Model
Model: SelfSupModel
DepthNet: PackNet01
PoseNet: PoseNet
Found 158 files


In [49]:
position = np.zeros(3)
orientation = np.eye(3)

for key in sorted(poses.keys()):
    rot_matrix, translation = poses[key]
    rot_matrix = rot_matrix.reshape(3,3)
    orientation = orientation.dot(rot_matrix.tolist())
    position += orientation.dot(translation.tolist())
    poses[key] = {"rot": rot_matrix.tolist(),
                  "trnas": translation.tolist(),
                  "poses": [*orientation[0], position[0], 
                            *orientation[1], position[1],
                            *orientation[2], position[2],
                            0, 0, 0, 1]}



In [52]:
json.dump(poses, open(outputs,"w"), sort_keys=True)
print(f"Written pose {len(poses)} of images to {outputs}")

Written pose 158 of images to ../Kitti/3_pose/pose.json


In [53]:
len(poses)

156